In [107]:
import Pkg; Pkg.add("Tables")
using CSV, Tables
using JuMP
using Gurobi

import Pkg; Pkg.add("JSON3")
using JSON3

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [108]:
const GRB_ENV = Gurobi.Env(output_flag=1);

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-04


#### Load in Data

In [109]:
all_classes = open("data/all_classes_info.json", "r") do file 
    JSON3.read(file)
end

R = CSV.File("data/ratings.csv",header=0) |> Tables.matrix;
H = CSV.File("data/hours.csv",header=0) |> Tables.matrix;
U = CSV.File("data/units.csv",header=0) |> Tables.matrix;

vars = CSV.File("data/variables.csv",header=0) |> Tables.matrix;
n = vars[1];
S = vars[2];

In [110]:
random_class = all_classes[rand(1:length(all_classes))]

JSON3.Object{Base.CodeUnits{UInt8, String}, SubArray{UInt64, 1, Vector{UInt64}, Tuple{UnitRange{Int64}}, true}} with 26 entries:
  :rating             => 4.95
  :has_final          => false
  :description        => "Practical exploration of community revitalization in …
  :offered_fall       => false
  :offered_spring     => true
  :instructors        => ["L. Hafrey, C. McDowell"]
  :joint_subjects     => ["15.679"]
  :out_of_class_hours => 4
  :total_units        => 9
  :related_subjects   => ["11.236", "11.360", "11.472", "11.025", "11.041", "11…
  :pdf_option         => false
  :in_class_hours     => 3
  :is_half_class      => false
  :level              => "G"
  :url                => "http://student.mit.edu/catalog/m11c.html#11.651"
  :subject_id         => "11.651"
  :title              => "USA Lab: Bridging the American Divides"
  :lab_units          => 1
  :design_units       => 0
  :public             => true
  :offered_summer     => false
  :lecture_units      => 3
  :prepara

In [111]:
subjectid_to_index = Dict(class[:subject_id] => ind for (ind, class) in enumerate(all_classes))
index_to_subjectid = Dict(ind => class[:subject_id] for (ind, class) in enumerate(all_classes))

Dict{Int64, String} with 4703 entries:
  4700 => "WGS.303"
  4576 => "STS.023"
  2288 => "21G.906"
  1703 => "18.S096"
  1956 => "20.102"
  2350 => "21H.227"
  3406 => "6.2061"
  2841 => "24.235"
  2876 => "24.908"
  687  => "12.751"
  185  => "1.THG"
  1090 => "15.5181"
  2015 => "20.561"
  3293 => "5.03"
  1704 => "18.S097"
  3220 => "4.609"
  422  => "11.353"
  1266 => "16.06"
  183  => "1.S992"
  1823 => "2.671"
  4030 => "9.822"
  551  => "12.005"
  3324 => "5.43"
  2119 => "21G.057"
  3298 => "5.067"
  ⋮    => ⋮

#### Defining the Basic Optimization Problem

In [112]:
model = Model(() -> Gurobi.Optimizer(GRB_ENV))

A JuMP Model
├ solver: Gurobi
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [113]:
@variable(model, X[1:n, 1:S], Bin)

4703×8 Matrix{VariableRef}:
 X[1,1]     X[1,2]     X[1,3]     …  X[1,6]     X[1,7]     X[1,8]
 X[2,1]     X[2,2]     X[2,3]        X[2,6]     X[2,7]     X[2,8]
 X[3,1]     X[3,2]     X[3,3]        X[3,6]     X[3,7]     X[3,8]
 X[4,1]     X[4,2]     X[4,3]        X[4,6]     X[4,7]     X[4,8]
 X[5,1]     X[5,2]     X[5,3]        X[5,6]     X[5,7]     X[5,8]
 X[6,1]     X[6,2]     X[6,3]     …  X[6,6]     X[6,7]     X[6,8]
 X[7,1]     X[7,2]     X[7,3]        X[7,6]     X[7,7]     X[7,8]
 X[8,1]     X[8,2]     X[8,3]        X[8,6]     X[8,7]     X[8,8]
 X[9,1]     X[9,2]     X[9,3]        X[9,6]     X[9,7]     X[9,8]
 X[10,1]    X[10,2]    X[10,3]       X[10,6]    X[10,7]    X[10,8]
 X[11,1]    X[11,2]    X[11,3]    …  X[11,6]    X[11,7]    X[11,8]
 X[12,1]    X[12,2]    X[12,3]       X[12,6]    X[12,7]    X[12,8]
 X[13,1]    X[13,2]    X[13,3]       X[13,6]    X[13,7]    X[13,8]
 ⋮                                ⋱  ⋮                     
 X[4692,1]  X[4692,2]  X[4692,3]     X[4692,6]  X[

In [140]:
α = 1 # alpha between [0-1], 1 being fully optimizing on minimizing hours 
@objective(model, Min, sum(α*H[i]*X[i,s] - (1-α)*R[i]*X[i,s] for i=1:n,s=1:S));

In [141]:
# Full-time; at least 36 units per semester
@constraint(model, [s in 1:S], sum(U[i]*X[i, s] for i=1:n) >= 36);

# Wellbeing: no more than 112 hours per week
@constraint(model, [s in 1:S], sum(H[i]*X[i, s] for i=1:n) <= 112);

# Not repeatable: cannot take a class again
@constraint(model, [i in 1:n], sum(X[i, t] for t=1:S) <= 1);\

# Taken correct semester
@constraint(model, [s in 1:2:S, i in 1:n], X[i, s] <= all_classes[i][:offered_fall]);
@constraint(model, [s in 2:2:S, i in 1:n], X[i, s] <= all_classes[i][:offered_spring]);

# 2 CI-H or HW
@constraint(model, sum(
    (get(all_classes[i], :communication_requirement, "") in ["CI-H", "CI-HW"]) * X[i, s] for i=1:n, s=1:S) >= 2);

# 8 HASS
@constraint(model, sum(
    (get(all_classes[i], :hass_attribute, "") in ["HASS-H", "HASS-A", "HASS-S"]) * X[i, s] for i=1:n, s=1:S) >= 8);

# 1 of each HASS-H, A, S
@constraint(model, [hass_type in ["HASS-H", "HASS-A", "HASS-S"]], sum(
    (get(all_classes[i], :hass_attribute, "") in [hass_type]) * X[i, s] for i=1:n, s=1:S) >= 1);


#### Adding Institute Reqs ####

In [142]:
chem_gir_ids = ["3.091", "5.111", "5.112"]
phys_1_gir_ids = ["8.01", "8.011", "8.012", "8.01L"]
phys_2_gir_ids = ["8.02", "8.021", "8.022"]
math_1_gir_ids = ["18.01", "18.01A"]
math_2_gir_ids = ["18.02", "18.02A", "18.022"]
bio_gir_ids = ["7.012", "7.013", "7.014", "7.015", "7.016"]
gir_ids = [chem_gir_ids, phys_1_gir_ids, phys_2_gir_ids, math_1_gir_ids, math_2_gir_ids, bio_gir_ids]
gir_inds_types = [[findfirst(x -> x[:subject_id] == gir_id, all_classes) for gir_id in gir_type] for gir_type in gir_ids]

6-element Vector{Vector{Int64}}:
 [2962, 3303, 3304]
 [3856, 3857, 3858, 3859]
 [3860, 3861, 3862]
 [1536, 1537]
 [1538, 1540, 1539]
 [3752, 3753, 3754, 3755, 3756]

In [143]:
# GIR constraints
@constraint(model, [gir_type in gir_inds_types], sum(X[gir_ind, s] for gir_ind in gir_type, s in 1:S) == 1);

#### Adding 6-4 Constraints ####

In [144]:
# main requirements
ai_programming_ids = ["6.100A", "6.100L"];
ai_math_1_ids = ["6.1200"];
ai_math_2_ids = ["6.S084", "18.C06", "18.06"];
ai_math_3_ids = ["6.3700", "6.3800", "18.05"];
ai_foundation_ids = ["6.1010", "6.1210"];
ai_centers_ids = ["6.1220", "6.1400", "6.3000", "6.3100", "6.3260", "6.3720", "6.3900", "6.3950",
                "6.4110", "6.4120", "6.4400", "6.4590", "6.7201", "6.C35", "9.660"] # removed "6.C571" and "6.C01"
ai_ids = [ai_programming_ids, ai_math_1_ids, ai_math_2_ids, ai_math_3_ids, ai_foundation_ids, ai_centers_ids];  # DOESN'T INCLUDE AI_MATH_IDS YET
ai_inds_types = [[findfirst(x -> x[:subject_id] == ai_id, all_classes) for ai_id in ai_type] for ai_type in ai_ids];


# additional constraints (centers)
ai_data_center_ids = ["6.3720", "6.3900"]; # no "6.C01"
ai_model_center_ids = ["6.3000", "6.3100", "6.4110", "6.4400"];
ai_dec_center_ids = ["6.3100", "6.4110", "6.7201"];  # no "6.C571"
ai_comp_center_ids = ["6.1220", "6.1400", "6.4400", "6.7201"]; # no "6.C571"
ai_human_center_ids = ["6.3260", "6.3950", "6.4120", "6.4590", "6.C35", "9.660"];
ai_centers_ids_list = [ai_data_center_ids, ai_model_center_ids, ai_dec_center_ids, ai_comp_center_ids, ai_human_center_ids];
ai_centers_inds_types = [[findfirst(x -> x[:subject_id] == ai_id, all_classes) for ai_id in ai_center] for ai_center in ai_centers_ids_list];

In [145]:
# electives
ai_app_cim = ["6.4200", "6.4210", "6.8301", "6.8611"]
ai_aus = ["18.404", "6.3730", "6.4210", "6.5151", "6.5831", "6.5931", "6.7411", "6.8301", "6.8371", "6.8611", "6.8701", "6.8711", "6.8801"] # removed "6.3020"
# 2 additional from eecs list
ai_electives_ids_list = [ai_app_cim, ai_aus]
ai_elec_inds_types = [[findfirst(x -> x[:subject_id] == ai_id, all_classes) for ai_id in ai_type] for ai_type in ai_electives_ids_list]

# other additional constraints 
ai_eecs_cim2_ids = ["6.1800", "6.1850", "6.2040", "6.2050", "6.2060", "6.2061", "6.2220", "6.2221", "6.2370", "6.2410", "6.2600", "6.4200", "6.4210", "6.4590", "6.4860", "6.4880", "6.8301", "6.8611", "6.9030", "6.UAR", "6.UAT"]
ai_serc_ids = ["6.3900", "6.3950", "6.4590", "6.8301", "6.8611"] # removed 6.C40
ai_add_constraints_ids_list = [ai_eecs_cim2_ids, ai_serc_ids]
ai_add_constraints_inds_types = [[findfirst(x -> x[:subject_id] == ai_id, all_classes) for ai_id in ai_type] for ai_type in ai_add_constraints_ids_list];

In [146]:
# ai_inds_types, 6 elem vector

# 1 programming skills
@constraint(model, sum(X[ai_ind, s] for ai_ind in ai_inds_types[1], s in 1:S) == 1);

# 3 math
ai_math_types = [ai_inds_types[2], ai_inds_types[3], ai_inds_types[4]];
@constraint(model, [math_type in ai_math_types], sum(X[math_ind, s] for math_ind in math_type, s in 1:S) == 1);

# 2 foundation
@constraint(model, sum(X[ai_ind, s] for ai_ind in ai_inds_types[5], s in 1:S) == 2);

# 5 total center
@constraint(model, sum(X[ai_ind, s] for ai_ind in ai_inds_types[6], s in 1:S) >= 5);

# 1 per center
@constraint(model, [ai_center in ai_centers_inds_types], sum(X[ai_ind, s] for ai_ind in ai_center, s in 1:S) >= 1);

# 2 from cim2
@constraint(model, sum(X[ai_ind, s] for ai_ind in ai_add_constraints_inds_types[1], s in 1:S) >= 2);

# 1 from ai+d-serc
@constraint(model, sum(X[ai_ind, s] for ai_ind in ai_add_constraints_inds_types[2], s in 1:S) >= 1);

# 1 elec from application cim and 1 from ai+d-aus
@constraint(model, [ai_elec_type in ai_elec_inds_types], sum(X[ai_ind, s] for ai_ind in ai_elec_type, s in 1:S) >= 1);

# 2 elecs ADDITIONAL from eecs or 18
# TODO

#### Adding Prereq constraints

In [147]:
raw_prerequisites = map(course -> get(course, "prerequisites", nothing), all_classes)

4703-element Vector{Union{Nothing, String}}:
 "GIR:CAL1"
 nothing
 "GIR:CAL1"
 nothing
 nothing
 nothing
 nothing
 "GIR:CAL2"
 nothing
 "''Permission of instructor''"
 "2.003, 2.016, 2.678"
 nothing
 nothing
 ⋮
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 "''One intermediate subject in French''/''permission of instructor''"
 nothing

In [148]:
abstract type ExprNode end

struct CourseNode <: ExprNode
    course::String
end

struct ANDNode <: ExprNode
    left::ExprNode
    right::ExprNode
end

struct ORNode <: ExprNode
    left::ExprNode
    right::ExprNode
end

struct PreReq
    root::CourseNode
    prereqs::Union{ExprNode, Nothing}
end 

In [149]:
function parse_prereq_string(input)
    if input === nothing
        return
    end
    input = replace(input, r"\s+" => "")
    tokens = collect(input)
    function parse(tokens::Vector{Char})::ExprNode
        stack = ExprNode[]
        buffer = IOBuffer()
        while !isempty(tokens)
            token = popfirst!(tokens)
            if token == '('
                node = parse(tokens)
                push!(stack, node)
            elseif token == ')'
                break
            elseif token == ','
                left = isempty(stack) ? CourseNode(String(take!(buffer))) : pop!(stack)
                right = parse(tokens)
                return ANDNode(left, right)
            elseif token == '/'
                left = isempty(stack) ? CourseNode(String(take!(buffer))) : pop!(stack)
                right = parse(tokens)
                return ORNode(left, right)
            else
                write(buffer, token)
            end
        end
        return isempty(stack) ? CourseNode(String(take!(buffer))) : stack[1]
    end
    return parse(tokens)
end

parse_prereq_string (generic function with 1 method)

In [150]:
function replace_gir_terms(input_str)
    if input_str === nothing
        return 
    end
    gir_map = Dict(
        "GIR:CHEM" => ["3.091", "5.111", "5.112"],
        "GIR:PHY1" => ["8.01", "8.011", "8.012", "8.01L"],
        "GIR:PHY2" => ["8.02", "8.021", "8.022"],
        "GIR:CAL1" => ["18.01", "18.01A"],
        "GIR:CAL2" => ["18.02", "18.02A", "18.022"],
        "GIR:BIOL" => ["7.012", "7.013", "7.014", "7.015", "7.016"]
    )
    pattern = r"GIR:\w+"
    replaced_str = replace(input_str, pattern => x -> begin
        term = match(pattern, x).match
        if haskey(gir_map, term)
            "(" * join(gir_map[term], "/") * ")"
        else
            term
        end
    end)
    return replaced_str
end

replace_gir_terms (generic function with 1 method)

In [151]:
prerequisites = map(prereq_str -> replace_gir_terms(prereq_str), raw_prerequisites)

4703-element Vector{Union{Nothing, String}}:
 "(18.01/18.01A)"
 nothing
 "(18.01/18.01A)"
 nothing
 nothing
 nothing
 nothing
 "(18.02/18.02A/18.022)"
 nothing
 "''Permission of instructor''"
 "2.003, 2.016, 2.678"
 nothing
 nothing
 ⋮
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 "''One intermediate subject in French''/''permission of instructor''"
 nothing

In [152]:
prerequisite_expressions = []
for index in 1:length(prerequisites)
    course_id = CourseNode(index_to_subjectid[index])
    prereq_tree = parse_prereq_string(prerequisites[index])
    push!(prerequisite_expressions, PreReq(course_id, prereq_tree))
end
prerequisite_expressions

4703-element Vector{Any}:
 PreReq(CourseNode("1.00"), ORNode(CourseNode("18.01"), CourseNode("18.01A")))
 PreReq(CourseNode("1.000"), nothing)
 PreReq(CourseNode("1.001"), ORNode(CourseNode("18.01"), CourseNode("18.01A")))
 PreReq(CourseNode("1.005"), nothing)
 PreReq(CourseNode("1.007"), nothing)
 PreReq(CourseNode("1.008"), nothing)
 PreReq(CourseNode("1.009"), nothing)
 PreReq(CourseNode("1.010"), ORNode(CourseNode("18.02"), ORNode(CourseNode("18.02A"), CourseNode("18.022"))))
 PreReq(CourseNode("1.011"), nothing)
 PreReq(CourseNode("1.013"), CourseNode("''Permissionofinstructor''"))
 PreReq(CourseNode("1.015"), ANDNode(CourseNode("2.003"), ANDNode(CourseNode("2.016"), CourseNode("2.678"))))
 PreReq(CourseNode("1.016"), nothing)
 PreReq(CourseNode("1.018"), nothing)
 ⋮
 PreReq(CourseNode("WGS.250"), nothing)
 PreReq(CourseNode("WGS.270"), nothing)
 PreReq(CourseNode("WGS.271"), nothing)
 PreReq(CourseNode("WGS.274"), nothing)
 PreReq(CourseNode("WGS.275"), nothing)
 PreReq(CourseNod

In [153]:
function add_course_constraint(course::CourseNode, model::Model, sem::Int)
    index = get(subjectid_to_index, course.course, nothing)
    if index == nothing
        return 0  # Ignore unknown courses
    end
    if sem == 1
        return 0  # Cannot satisfy prerequisites in the first semester
    else
        return handle_prerequisite_sum(index, model, sem)
    end
end

function handle_prerequisite_sum(index::Int, model::Model, sem::Int)
    prereq_sum = sum(X[index, s] for s in 1:sem-1)
    binary_prereq_var = @variable(model, binary=true)
    @constraint(model, binary_prereq_var <= prereq_sum)
    @constraint(model, binary_prereq_var >=  prereq_sum / (sem-1))  # Ensure binary behavior
    return binary_prereq_var
end

function add_and_constraint(left_var, right_var, model::Model)
    and_var = @variable(model, binary=true)
    @constraint(model, and_var <= left_var)
    @constraint(model, and_var <= right_var)
    @constraint(model, and_var >= left_var + right_var - 1)
    return and_var
end

function add_or_constraint(left_var, right_var, model::Model)
    or_var = @variable(model, binary=true)
    @constraint(model, or_var >= left_var)
    @constraint(model, or_var >= right_var)
    @constraint(model, or_var <= left_var + right_var)
    return or_var
end

function add_and_or_constraints(expr::ExprNode, model::Model, sem::Int)
    if isa(expr, CourseNode)
        return add_course_constraint(expr, model, sem)
    elseif isa(expr, ANDNode)
        left_var = add_and_or_constraints(expr.left, model, sem)
        right_var = add_and_or_constraints(expr.right, model, sem)
        return add_and_constraint(left_var, right_var, model)
    elseif isa(expr, ORNode)
        left_var = add_and_or_constraints(expr.left, model, sem)
        right_var = add_and_or_constraints(expr.right, model, sem)
        return add_or_constraint(left_var, right_var, model)
    else
        error("Unknown expression type")
    end
end

function add_prereq_constraint(prereq_obj::PreReq, model::Model, sem::Int)
    index = get(subjectid_to_index, prereq_obj.root.course, nothing)
    if index == nothing
        return 1  # Ignore unknown courses
    end
    if prereq_obj.prereqs === nothing
        return X[index, sem]
    else
        prereq_var = add_and_or_constraints(prereq_obj.prereqs, model, sem)
        @constraint(model, X[index, sem] <= prereq_var)
        return X[index, sem]
    end
end


add_prereq_constraint (generic function with 1 method)

In [154]:
### Adding Course prereq constraints
for s in 1:S
    for prereq_expr in prerequisite_expressions
        add_prereq_constraint(prereq_expr, model, s)
    end
end

#### Optimizing the Model and Interpretation

In [155]:
optimize!(model)

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 21.5.0 21F79)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 422334 rows, 155174 columns and 1359208 nonzeros
Model fingerprint: 0x50040df8
Variable types: 0 continuous, 155174 integer (155174 binary)
Coefficient statistics:
  Matrix range     [1e-01, 8e+01]
  Objective range  [1e+00, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]

MIP start from previous solve produced solution with objective 217.68 (0.18s)
Loaded MIP start from previous solve with objective 217.68

Presolve removed 377967 rows and 129679 columns
Presolve time: 2.47s
Presolved: 44367 rows, 25495 columns, 153568 nonzeros
Variable types: 0 continuous, 25495 integer (25495 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing primal log only...

Concurrent spin time: 0.03s

Solved with dual simplex

Use crossover to convert LP s

In [162]:
function interpret_course_nums(classes_id_per_sem)
    course_nums = [[all_classes[class_ind][:subject_id] for class_ind in sem] for sem in classes_id_per_sem]
    return course_nums
end



function interpret_course_attr(classes_id_per_sem, attr)
    courses_attr = [[get(all_classes[class_ind], Symbol(attr), "") for class_ind in sem] for sem in classes_id_per_sem]
    return courses_attr
end

function interpret_course_hours(classes_id_per_sem)
    course_hours = [[H[class_ind] for class_ind in sem] for sem in classes_id_per_sem]
    return course_hours
end

function interpret_course_units(classes_id_per_sem)
    course_units = [[U[class_ind] for class_ind in sem] for sem in classes_id_per_sem]
    return course_units
end

interpret_course_units (generic function with 1 method)

In [161]:
taken_classes_sem = [[ind for (ind, val) in enumerate(sem) if val == 1] for sem in eachcol(value.(X))]
interpret_course_nums(taken_classes_sem)

8-element Vector{Vector{String}}:
 ["21M.622", "21M.747", "8.01L"]
 ["18.01", "21M.460", "3.091", "6.100A"]
 ["18.022", "6.1010", "6.1200"]
 ["21M.623", "6.1210", "ES.113"]
 ["21M.451", "21W.032", "6.3700", "8.02"]
 ["18.06", "6.1400", "6.3260"]
 ["6.3900", "6.3950", "7.012"]
 ["6.3100", "6.8301", "6.UAT"]

In [166]:
function interpret_course_attr(classes_id_per_sem, "title")

LoadError: UndefVarError: `course_ids` not defined

In [159]:
prerequisite_expressions[subjectid_to_index["16.84"]]

PreReq(CourseNode("16.84"), ORNode(CourseNode("6.4200"), CourseNode("''permissionofinstructor''")))

In [160]:
sems_hours = interpret_course_hours(taken_classes_sem)
[sum(sem_hours) for sem_hours in sems_hours]

8-element Vector{Float64}:
 20.32
 31.049999999999997
 31.43
 20.43
 31.09
 28.15
 25.86
 29.349999999999998